In [1]:
from typing import Dict, Optional, Any, List, Self, Literal, ClassVar, Union
from uuid import UUID, uuid4
from pydantic import BaseModel, Field, model_validator, computed_field, field_validator
from enum import Enum

from dnd.blocks import (
    BaseBlock, AbilityScores, SavingThrowSet, Health, 
    Equipped, Speed, ActionEconomy,SkillSet,skill_names
)
from dnd.values import ModifiableValue
from dnd.modifiers import (
    NumericalModifier, DamageType, ResistanceStatus, 
    ContextAwareCondition, BaseObject
)

from dnd.equipment import (
    Armor, Weapon, Shield, BodyArmor, Gauntlets, Greaves,
    Boots, Amulet, Ring, Cloak, Helmet, BodyPart
)
from dnd.dice import Dice, RollType, DiceRoll

In [2]:
class Entity(BaseBlock):
    """ Base class for dnd entities in the game it acts as container for blocks and implements common functionalities that
    require interactions between blocks """
    
    name: str = Field(default="Entity")
    ability_scores: AbilityScores = Field(default_factory=lambda: AbilityScores.create(source_entity_uuid=uuid4()))
    skill_set: SkillSet = Field(default_factory=lambda: SkillSet.create(source_entity_uuid=uuid4()))
    saving_throws: SavingThrowSet = Field(default_factory=lambda: SavingThrowSet.create(source_entity_uuid=uuid4()))
    health: Health = Field(default_factory=lambda: Health.create(source_entity_uuid=uuid4()))
    equipped: Equipped = Field(default_factory=lambda: Equipped.create(source_entity_uuid=uuid4()))
    speed: Speed = Field(default_factory=lambda: Speed.create(source_entity_uuid=uuid4()))
    action_economy: ActionEconomy = Field(default_factory=lambda: ActionEconomy.create(source_entity_uuid=uuid4()))
    proficiency_bonus: ModifiableValue = Field(default_factory=lambda: ModifiableValue.create(source_entity_uuid=uuid4(),value_name="proficiency_bonus",base_value=0))

    



In [3]:
goblin = Entity.create(source_entity_uuid=uuid4(),name="Goblin")
assert isinstance(goblin, Entity)


Health(name='Base Block', uuid=UUID('ec6d5ae7-d0c8-42a8-9ea4-d6fae741dff4'), source_entity_uuid=UUID('5af6d614-b097-4efa-a44f-e7abbf96ebe5'), source_entity_name=None, target_entity_uuid=None, target_entity_name=None, context=None, hit_dices=[HitDice(name='HitDice', uuid=UUID('bf47277f-8840-43f6-953d-ba59682a5fdb'), source_entity_uuid=UUID('99ad46f0-243c-47e2-b4cd-e9eb33cea1b5'), source_entity_name=None, target_entity_uuid=None, target_entity_name=None, context=None, hit_dice_value=ModifiableValue(name='Hit Dice Value', uuid=UUID('a570a94f-c085-462f-941a-4536170fa916'), source_entity_uuid=UUID('99ad46f0-243c-47e2-b4cd-e9eb33cea1b5'), source_entity_name=None, target_entity_uuid=None, target_entity_name=None, context=None, score_normalizer=<function ModifiableValue.<lambda> at 0x0000025CFD4D1580>, generated_from=[], self_static=StaticValue(name='A Value', uuid=UUID('528500c7-8655-4cb0-8e76-0c1d32acb854'), source_entity_uuid=UUID('99ad46f0-243c-47e2-b4cd-e9eb33cea1b5'), source_entity_name=